Mining artciles from the internet

In [ ]:
!pip install newspaper3k
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 1

upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    
    try:
       
        page=requests.get(url)                            
    
    except Exception as e:                                  
        error_type, error_obj, error_info = sys.exc_info()       
        print ('ERROR FOR LINK:',url)                           
        print (error_type, 'Line:', error_info.tb_lineno)      
        continue                                              
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    filename="NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
dfl=data.head()
data.columns

     |████████████████████████████████| 215kB 5.7MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 92kB 5.7MB/s 
     |████████████████████████████████| 7.4MB 13.1MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=ca65302867b894a2560539aff3a82921a7dd880f98dbe70416ff7857e57d2e2c
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3355 sha256=930c3dfdd80c521f02abe9bf1e92b841a900834d1bc16a877b8560ba674cd68e
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=06d0eeffe84aaad2b221f774ad21bb9fa0b3e4dd9ae163b59eee193b754a7697
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35d

Index(['Statement', 'Link', 'Date', 'Source', 'Label'], dtype='object')

In [ ]:
from newspaper import Article
import nltk
nltk.download('punkt')
articles=[]
for i in range(5):
  articles.append(data.Link[i])
print(articles)
a_text=[]
Titles = []
for l in articles:
  article = Article(l, language="en")
  article.download()                      
  article.parse()                         
  article.nlp()                           
  a_text.append(article.text)           
  Titles.append(article.title)  




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['https://www.politifact.com/factchecks/2020/nov/23/tim-murtaugh/tim-murtaugh-wrongly-says-key-pennsylvania-mail-ba/', 'https://www.politifact.com/factchecks/2020/nov/21/joe-sanfelippo/voters-not-clerks-decided-who-was-indefinitely-con/', 'https://www.politifact.com/factchecks/2020/nov/20/sidney-powell/sidney-powell-claim-450000-votes-were-only-biden-k/', 'https://www.politifact.com/factchecks/2020/nov/20/rudy-giuliani/giuliani-cites-affidavit-crucial-errors-press-conf/', 'https://www.politifact.com/factchecks/2020/nov/20/joe-biden/biden-mischaracterizes-teacher-layoffs-pandemic/']


Preprocessing of the article data


In [ ]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from collections import Counter
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
def convert_lower_case(data):
    return np.char.lower(data)
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    data = np.char.replace(data, "'", "")
    return data
def lemmatization(data):
    lemmatizer = WordNetLemmatizer() 
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text
def preprocess(data):
    data = convert_lower_case(data)     
    data = remove_punctuation(data)       
    data = remove_stop_words(data)        
    data = lemmatization(data)            
    return data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
n_atext = []

for i in range(5):

  n_article = preprocess(a_text[i])

  n_atext.append(n_article)

In [ ]:
article_ids = ["Article_" + str(i) for i in range(5)]

article_dict = dict(zip(article_ids, Titles))

ids = list(article_dict.keys())

pairs = []
for i, v in enumerate(ids):
    for j in ids[i+1:]:
        pairs.append((ids[i], j))

Calculating Cosine similarity between articles

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,1))
vectorizer
feature_matrix = vectorizer.fit_transform(n_atext).astype(float)
def compute_cosine_similarity(pair):
    
    article1, article2 = pair
    
    article1_index = int(article1.split("_")[1])
    article2_index = int(article2.split("_")[1])
    
    article1_fm = feature_matrix.toarray()[article1_index]
    article2_fm = feature_matrix.toarray()[article2_index]
    
    manual_cosine_similarity = np.dot(article1_fm, article2_fm)
    
    return manual_cosine_similarity
cosine_similarity = [compute_cosine_similarity(pair) for pair in pairs]
df = pd.DataFrame({'pair': pairs, 'similarity': cosine_similarity})
df.head()

,pair,similarity
0,"(Article_0, Article_1)",0.112471
1,"(Article_0, Article_2)",0.177160
2,"(Article_0, Article_3)",0.111870
3,"(Article_0, Article_4)",0.029453
4,"(Article_1, Article_2)",0.163870
